## Validation 和 Cross Validation

In [78]:
import numpy as np
from sklearn import datasets

In [79]:
digits = datasets.load_digits()
X = digits.data
y = digits.target

### 测试train_test_split

In [80]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=666)

In [81]:
%%time
from sklearn.neighbors import KNeighborsClassifier

best_k, best_p, best_score = 0, 0, 0
for k in range(2, 11):
    for p in range(1, 6):
        knn_clf = KNeighborsClassifier(weights="distance", n_neighbors=k, p=p)
        knn_clf.fit(X_train, y_train)
        score = knn_clf.score(X_test, y_test)
        if score > best_score:
            best_k, best_p, best_score = k, p, score
            
print("Best K =", best_k)
print("Best P =", best_p)
print("Best Score =", best_score)

Best K = 3
Best P = 4
Best Score = 0.986091794159
CPU times: user 29 s, sys: 62.8 ms, total: 29.1 s
Wall time: 29.2 s


### 使用交叉验证

In [92]:
knn_clf = KNeighborsClassifier()
cross_val_score(knn_clf, X_train, y_train)

array([ 0.98895028,  0.97777778,  0.96629213])

In [82]:
%%time
from sklearn.model_selection import cross_val_score

best_k, best_p, best_score = 0, 0, 0
for k in range(2, 11):
    for p in range(1, 6):
        knn_clf = KNeighborsClassifier(weights="distance", n_neighbors=k, p=p)
        scores = cross_val_score(knn_clf, X_train, y_train)
        score = np.mean(scores)
        if score > best_score:
            best_k, best_p, best_score = k, p, score
            
print("Best K =", best_k)
print("Best P =", best_p)
print("Best Score =", best_score)

Best K = 2
Best P = 2
Best Score = 0.982359987401
CPU times: user 30.8 s, sys: 66.2 ms, total: 30.9 s
Wall time: 31 s


In [85]:
best_knn_clf = KNeighborsClassifier(weights="distance", n_neighbors=2, p=2)
best_knn_clf.fit(X_train, y_train)
best_knn_clf.score(X_test, y_test)

0.98052851182197498

### 回顾网格搜索

In [86]:
%%time
from sklearn.model_selection import GridSearchCV

param_grid = [
    {
        'weights': ['distance'],
        'n_neighbors': [i for i in range(2, 11)], 
        'p': [i for i in range(1, 6)]
    }
]

grid_search = GridSearchCV(knn_clf, param_grid, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 45 candidates, totalling 135 fits
CPU times: user 1min 31s, sys: 399 ms, total: 1min 31s
Wall time: 1min 32s


[Parallel(n_jobs=1)]: Done 135 out of 135 | elapsed:  1.5min finished


In [87]:
grid_search.best_score_

0.98237476808905377

In [88]:
grid_search.best_params_

{'n_neighbors': 2, 'p': 2, 'weights': 'distance'}

In [89]:
best_knn_clf = grid_search.best_estimator_
best_knn_clf.score(X_test, y_test)

0.98052851182197498

### cv参数

In [90]:
cross_val_score(knn_clf, X_train, y_train, cv=5)

array([ 0.99086758,  0.98173516,  0.98611111,  0.98130841,  0.98571429])

In [91]:
grid_search = GridSearchCV(knn_clf, param_grid, verbose=1, cv=5)